# Pre-processing Nebraska Allocation data for WaDEQA upload.
Date Updated: 05/25/2023
Purpose:  To pre-process the Nebraska data into one master file for simple DataFrame creation and extraction

### Notes:
- asfd

In [ ]:
#Needed Libararies

# working with data
import os
import numpy as np
import pandas as pd
import geopandas as gpd

# visulizaiton
import matplotlib.pyplot as plt
import seaborn as sns

# API retrieval
import requests
import json

# Cleanup
import re
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook
pd.set_option('display.float_format', lambda x: '%.5f' % x) # suppress scientific notation in Pandas

In [ ]:
#Working Directory
workingDir = "G:/Shared drives/WaDE Data/Nebraska/WaterAllocation/RawInputData"
os.chdir(workingDir)

## Input Data

### Retreiev api data

In [ ]:
# %%time
# # API retrieval
# # shoot for 30 pages, that seemed to be all that AllSurfaceWaterPoints offered.
# df = pd.DataFrame()
# countPage = 1
# while countPage < 30:
#     url = "https://nednr.nebraska.gov/IwipApi/api/v1/WaterRights/AllSurfaceWaterPoints?page=" + str(countPage)
#     print(url)

#     # store in dataframe
#     try:
#         responseD = json.loads(requests.get(url).text)
#         DtL = responseD['Results']
#         length = len(DtL)
#         for i in range(length):
#             row = pd.DataFrame([DtL[i]])
#             df = pd.concat([df, row])
#     except:
#         print("Error, issue with API return.")
    
#     countPage = countPage + 1

# # # Use only NeNDR Active provided sites
# # df = df[df['SourceName'] == 'NeDNR']

# print(len(df))
# df.head()

In [ ]:
# # explode these list....
# dftemp = df.copy()
# dftemp = dftemp.explode('NoticeExemptions')
# dftemp = dftemp.explode('Notices')
# dftemp = dftemp.explode('PointOfDiversions')
# dftemp = dftemp.explode('Contacts')

# print(len(dftemp))
# dftemp.head(1)

In [ ]:
# # To unpack column's dictionary value new into separate columns -> contact to existing dataframe -> drop unpacked column
# dftemp = pd.concat([dftemp, dftemp["RightUse"].apply(pd.Series)], axis=1).drop(columns="RightUse")
# dftemp = pd.concat([dftemp, dftemp["NoticeExemptions"].apply(pd.Series)], axis=1).drop(columns="NoticeExemptions")
# dftemp = pd.concat([dftemp, dftemp["Notices"].apply(pd.Series)], axis=1).drop(columns="Notices")
# dftemp = pd.concat([dftemp, dftemp["PointOfDiversions"].apply(pd.Series)], axis=1).drop(columns="PointOfDiversions")
# dftemp = pd.concat([dftemp, dftemp["Contacts"].apply(pd.Series)], axis=1).drop(columns="Contacts")

# print(len(dftemp))
# dftemp.head(1)

In [ ]:
# # Clean Data
# # we don't really need the 'NoticeExemptions' at this time
# dropList = ['PumpSheets', 'SpecialConditions', 'NoticeID', 'NoticeType', 'NoticeDate', 'EffectiveDate', 'ReasonForAdminAction', 'Notes', 'DeleteNotice']
# dftemp = dftemp.drop(dropList, axis=1).drop_duplicates().reset_index(drop=True)
# #dftemp = dftemp.drop(dropList, axis=1)
# print(len(dftemp))
# dftemp.head()

In [ ]:
# # export api data
# dftemp.to_csv('AllSurfaceWaterPoints.zip', compression=dict(method='zip', archive_name='AllSurfaceWaterPoints.csv'), index=False)  # The output, save as a zip

# POD Data

In [ ]:
# POD Data
# read in the above API data that was saved to hard file
PoDAAInput = "AllSurfaceWaterPoints.zip"
dfPoD = pd.read_csv(PoDAAInput).replace(np.nan, "")

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in dfPoD:
    dfPoD['WaDEUUID'] = "neD" + dfPoD.index.astype(str)
    dfPoD.to_csv('AllSurfaceWaterPoints.zip', compression=dict(method='zip', archive_name='AllSurfaceWaterPoints.csv'), index=False)
    
print(len(dfPoD))
dfPoD.head(1)

In [ ]:
# Drop non-Active AllocationLegalStatusCV Water Rights
# We only want Active water rights
# We only want active POD sites
dfPoD = dfPoD[dfPoD['RightStatus'] == 'Active'].reset_index(drop=True)
dfPoD = dfPoD[dfPoD['PODStatus'] == 'Active'].reset_index(drop=True)
print(len(dfPoD))
dfPoD.head(1)

In [ ]:
dfPoD['RightStatus'].unique()

In [ ]:
dfPoD['PODStatus'].unique()

In [ ]:
# assign VariableSpecificUUID

def assignVariableSpecificUUID(colvA):
    outList = ''
    colvA = str(colvA).strip()
    if colvA == "CFS":
        outList = "NEwr_V1"
    if colvA == "AF":
        outList = "NEwr_V2"

    return outList

dfPoD['in_VariableSpecificUUID'] = dfPoD.apply(lambda row: assignVariableSpecificUUID(row['Units']), axis=1)
dfPoD['in_VariableSpecificUUID'].unique()

In [ ]:
# AllocationFlow_CFS - based on reporeted Unit

def assignAllocationFlow_CFS(colvA, colvB):
    if colvA == '' or pd.isnull(colvA):
        outList = ''
    else:
        colvB = colvB.strip()
        if colvB == "CFS":
            outList = colvA
        else:
            outList = ''
    return outList

dfPoD['AllocationFlow_CFS'] = dfPoD.apply(lambda row: assignAllocationFlow_CFS(row['ProGrant'], row['Units']), axis=1)
dfPoD['AllocationFlow_CFS'].unique()

In [ ]:
# AllocationVolume_AF - based on reporeted Unit

def assignAllocationVolume_AF(colvA, colvB):
    if colvA == '' or pd.isnull(colvA):
        outList = ''
    else:
        colvB = colvB.strip()
        if colvB == "AF":
            outList = colvA
        else:
            outList = ''
    return outList

dfPoD['AllocationVolume_AF'] = dfPoD.apply(lambda row: assignAllocationVolume_AF(row['ProGrant'], row['Units']), axis=1)
dfPoD['AllocationVolume_AF'].unique()

In [ ]:
dfPoD['WaterAllocationNativeURL'] = 'https://nednr.nebraska.gov/dynamic/WaterRights/WaterRights/SWRDetailPage?RightId=' + dfPoD['RightID'].astype(str)
dfPoD.head(1)

In [ ]:
# create output POD dataframe
df = pd.DataFrame()

# Data Assessment UUID
df['WaDEUUID'] = dfPoD['WaDEUUID']

# # Method Info
df['in_MethodUUID'] = "NEwr_M1"

# Variable Info
df['in_VariableSpecificUUID'] = dfPoD['in_VariableSpecificUUID']

# Organization Info
df['in_OrganizationUUID'] = "NEwr_O1"

# WaterSource Info
df['in_Geometry'] = ""
df['in_GNISFeatureNameCV'] = ""
df['in_WaterQualityIndicatorCV'] = ""
df['in_WaterSourceName'] = dfPoD['SourceName']
df['in_WaterSourceNativeID'] = "" # auto fill in below
df['in_WaterSourceTypeCV'] = "Surface Water"

# Site Info
df['in_CoordinateAccuracy'] = ""
df['in_CoordinateMethodCV'] = ""
df['in_County'] = ""
df['in_EPSGCodeCV'] = 4326
df['in_Geometry'] = ""
df['in_GNISCodeCV'] = ""
df['in_HUC12'] = dfPoD['HUC12']
df['in_HUC8'] = ""
df['in_Latitude'] = dfPoD['LatitudeDecimalDegrees']
df['in_Longitude'] = dfPoD['LongitudeDecimalDegrees']
df['in_NHDNetworkStatusCV'] = ""
df['in_NHDProductCV'] = ""
df['in_PODorPOUSite'] = "POD"
df['in_SiteName'] = ""
df['in_SiteNativeID'] = "POD" + dfPoD['PointOfDiversionID'].replace("", 0).fillna(0).astype(int).astype(str)
df['in_SitePoint'] = ""
df['in_SiteTypeCV'] = ""
df['in_StateCV'] = "NE"
df['in_USGSSiteID'] = ""

# AllocationAmount Info
df['in_AllocationApplicationDate'] = ""
df['in_AllocationAssociatedConsumptiveUseSiteIDs'] = ""
df['in_AllocationAssociatedWithdrawalSiteIDs'] = ""
df['in_AllocationBasisCV'] = ""
df['in_AllocationChangeApplicationIndicator'] = ""
df['in_AllocationCommunityWaterSupplySystem'] = ""
df['in_AllocationCropDutyAmount'] = ""
df['in_AllocationExpirationDate'] = ""
df['in_AllocationFlow_CFS'] = dfPoD['AllocationFlow_CFS']
df['in_AllocationLegalStatusCV'] = dfPoD['RightStatus']
df['in_AllocationNativeID'] =  dfPoD['RightID'].replace("", 0).fillna(0).astype(str).str.lower().str.strip()
df['in_AllocationOwner'] = dfPoD['FirstName'] + " " + dfPoD['LastName']
df['in_AllocationPriorityDate'] = dfPoD['PriorityDate']
df['in_AllocationSDWISIdentifierCV'] = ""
df['in_AllocationTimeframeEnd'] = ""
df['in_AllocationTimeframeStart'] = ""
df['in_AllocationTypeCV'] = ""
df['in_AllocationVolume_AF'] = dfPoD['AllocationVolume_AF']
df['in_BeneficialUseCategory'] = dfPoD['UseDescription']
df['in_CommunityWaterSupplySystem'] = ""
df['in_CropTypeCV'] = ""
df['in_CustomerTypeCV'] = ""
df['in_DataPublicationDate'] = ""
df['in_DataPublicationDOI'] = ""
df['in_ExemptOfVolumeFlowPriority'] = 0
df['in_GeneratedPowerCapacityMW'] = ""
df['in_IrrigatedAcreage'] = ""
df['in_IrrigationMethodCV'] = ""
df['in_LegacyAllocationIDs'] = ""
df['in_OwnerClassificationCV'] = ""
df['in_PopulationServed'] = ""
df['in_PowerType'] = ""
df['in_PrimaryBeneficialUseCategory'] = ""
df['in_SDWISIdentifierCV'] = ""
df['in_WaterAllocationNativeURL'] = dfPoD['WaterAllocationNativeURL']

outdfPoD = df.copy()
outdfPoD = outdfPoD.drop_duplicates().reset_index(drop=True)
print(len(outdfPoD))
outdfPoD.head()

# POU Data

In [ ]:
# Input File - PoU Shapefile Data
# export dataframe as zipped csv
pouInput = 'shapefile/BND_SurfaceWaterRights_DNR.zip'
dfPOU = gpd.read_file(pouInput).replace(np.nan, "").replace("nan,nan", "") #geodataframe read
dfPOU = dfPOU.drop(['geometry'], axis=1)

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in dfPOU:
    dfPOU['WaDEUUID'] = "waU" + dfPOU.index.astype(str)
    dfPOU.to_csv('BND_SurfaceWaterRights_DNR.zip', compression=dict(method='zip', archive_name='BND_SurfaceWaterRights_DNR.csv'), index=False)

print(len(dfPOU))
dfPOU.head(1)

In [ ]:
# assign VariableSpecificUUID

def assignVariableSpecificUUID(colvA):
    outList = ''
    colvA = str(colvA).strip()
    if colvA == "CFS":
        outList = "NEwr_V1"
    if colvA == "AF":
        outList = "NEwr_V2"

    return outList

dfPOU['in_VariableSpecificUUID'] = dfPOU.apply(lambda row: assignVariableSpecificUUID(row['Units']), axis=1)
dfPOU['in_VariableSpecificUUID'].unique()

In [ ]:
# AllocationFlow_CFS - based on reporeted Unit

def assignAllocationFlow_CFS(colvA, colvB):
    if colvA == '' or pd.isnull(colvA):
        outList = ''
    else:
        colvB = colvB.strip()
        if colvB == "CFS":
            outList = colvA
        else:
            outList = ''
    return outList

dfPOU['AllocationFlow_CFS'] = dfPOU.apply(lambda row: assignAllocationFlow_CFS(row['ProGrant'], row['Units']), axis=1)
dfPOU['AllocationFlow_CFS'].unique()

In [ ]:
# AllocationVolume_AF - based on reporeted Unit

def assignAllocationVolume_AF(colvA, colvB):
    if colvA == '' or pd.isnull(colvA):
        outList = ''
    else:
        colvB = colvB.strip()
        if colvB == "AF":
            outList = colvA
        else:
            outList = ''
    return outList

dfPOU['AllocationVolume_AF'] = dfPOU.apply(lambda row: assignAllocationVolume_AF(row['ProGrant'], row['Units']), axis=1)
dfPOU['AllocationVolume_AF'].unique()

In [ ]:
dfPOU['WaterAllocationNativeURL'] = 'https://nednr.nebraska.gov/dynamic/WaterRights/WaterRights/SWRDetailPage?RightId=' + dfPOU['RightID'].astype(str)
dfPOU.head(1)

In [ ]:
dfPOU['RightUse'].unique()

In [ ]:
# create output POD dataframe
df = pd.DataFrame()

# Data Assessment UUID
df['WaDEUUID'] = dfPOU['WaDEUUID']

# # Method Info
df['in_MethodUUID'] = "NEwr_M1"

# Variable Info
df['in_VariableSpecificUUID'] = dfPOU['in_VariableSpecificUUID']

# Organization Info
df['in_OrganizationUUID'] = "NEwr_O1"

# WaterSource Info
df['in_Geometry'] = ""
df['in_GNISFeatureNameCV'] = ""
df['in_WaterQualityIndicatorCV'] = ""
df['in_WaterSourceName'] = dfPOU['SourceName']
df['in_WaterSourceNativeID'] = "" # auto fill in below
df['in_WaterSourceTypeCV'] = "Surface Water"

# Site Info
df['in_CoordinateAccuracy'] = ""
df['in_CoordinateMethodCV'] = ""
df['in_County'] = ""
df['in_EPSGCodeCV'] = 4326
df['in_Geometry'] = ""
df['in_GNISCodeCV'] = ""
df['in_HUC12'] = dfPOU['HUC12']
df['in_HUC8'] = ""
df['in_Latitude'] = dfPOU['Latitude']
df['in_Longitude'] = dfPOU['Longitude']
df['in_NHDNetworkStatusCV'] = ""
df['in_NHDProductCV'] = ""
df['in_PODorPOUSite'] = "POU"
df['in_SiteName'] = ""
df['in_SiteNativeID'] = "POU" + dfPOU['wadeID'].replace("", 0).fillna(0).astype(str)
df['in_SitePoint'] = ""
df['in_SiteTypeCV'] = ""
df['in_StateCV'] = "NE"
df['in_USGSSiteID'] = ""

# AllocationAmount Info
df['in_AllocationApplicationDate'] = ""
df['in_AllocationAssociatedConsumptiveUseSiteIDs'] = ""
df['in_AllocationAssociatedWithdrawalSiteIDs'] = ""
df['in_AllocationBasisCV'] = ""
df['in_AllocationChangeApplicationIndicator'] = ""
df['in_AllocationCommunityWaterSupplySystem'] = ""
df['in_AllocationCropDutyAmount'] = ""
df['in_AllocationExpirationDate'] = ""
df['in_AllocationFlow_CFS'] = dfPOU['AllocationFlow_CFS']
df['in_AllocationLegalStatusCV'] = ""
df['in_AllocationNativeID'] =  dfPOU['RightID'].replace("", 0).fillna(0).astype(str).str.lower().str.strip()
df['in_AllocationOwner'] = ""
df['in_AllocationPriorityDate'] = dfPOU['PriorityDa']
df['in_AllocationSDWISIdentifierCV'] = ""
df['in_AllocationTimeframeEnd'] = ""
df['in_AllocationTimeframeStart'] = ""
df['in_AllocationTypeCV'] = ""
df['in_AllocationVolume_AF'] = dfPOU['AllocationVolume_AF']
df['in_BeneficialUseCategory'] = dfPOU['RightUse']
df['in_CommunityWaterSupplySystem'] = ""
df['in_CropTypeCV'] = ""
df['in_CustomerTypeCV'] = ""
df['in_DataPublicationDate'] = ""
df['in_DataPublicationDOI'] = ""
df['in_ExemptOfVolumeFlowPriority'] = 0
df['in_GeneratedPowerCapacityMW'] = ""
df['in_IrrigatedAcreage'] = ""
df['in_IrrigationMethodCV'] = ""
df['in_LegacyAllocationIDs'] = ""
df['in_OwnerClassificationCV'] = ""
df['in_PopulationServed'] = ""
df['in_PowerType'] = ""
df['in_PrimaryBeneficialUseCategory'] = ""
df['in_SDWISIdentifierCV'] = ""
df['in_WaterAllocationNativeURL'] = dfPOU['WaterAllocationNativeURL']

outdfPoU = df.copy()
outdfPoU = outdfPoU.drop_duplicates().reset_index(drop=True)
print(len(outdfPoU))
outdfPoU.head()

## Concatenate Data

In [ ]:
# Concatenate dataframes
frames = [outdfPoD, outdfPoU]
outdf = pd.concat(frames)
outdf = outdf.drop_duplicates().reset_index(drop=True).replace(np.nan, "")
print(len(outdf))

In [ ]:
# Making Sure datatype of HUC12 is int.

def assignHUC12(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ''
    else:
        outList = int(colrowValue)
    return outList

outdf['in_HUC12'] = outdf.apply(lambda row: assignHUC12(row['in_HUC12']), axis=1)
outdf['in_HUC12'].unique()

## Clean Data & WaDE Custom Elements

In [ ]:
# solving a long benefical use
def longBenUseTempFix(val):
    if val == "Supplemental Cooling (an app. for water for cooling through a system that has a prior app. for cooling)":
        outList = "Supplemental Cooling (app for water for cooling through a system that has a prior app for cooling)"
    elif val == "Supplemental Irrigation (irrig. from reservoir on lands also covered by a natural flow appropriation)":
        outList = "Supplemental Irrigation (irrig. from reservoir on lands covered by a natural flow appropriation)"
    else:
        outList = val
    return outList
outdf['in_BeneficialUseCategory'] = outdf.apply(lambda row: longBenUseTempFix(row['in_BeneficialUseCategory']), axis=1)

for x in outdf['in_BeneficialUseCategory'].sort_values().unique():
    print(f'"' + str(x) + '",')

In [ ]:
# Clean owner name up
def removeSpecialCharsFunc(Val):
    Val = str(Val)
    Val = re.sub("[$@&.;,/\)(-]", "", Val).title().replace("  ", " ").strip()
    return Val

In [ ]:
outdf['in_AllocationOwner'] = outdf.apply(lambda row: removeSpecialCharsFunc(row['in_AllocationOwner']), axis=1)
outdf['in_AllocationOwner'].unique()

In [ ]:
outdf['in_WaterSourceName'] = outdf.apply(lambda row: removeSpecialCharsFunc(row['in_WaterSourceName']), axis=1)
outdf['in_WaterSourceName'].unique()

In [ ]:
# Ensure Empty String

def ensureEmptyString(val):
    val = str(val).strip()
    if val == "" or val == " " or val == "nan" or pd.isnull(val):
        outString = ""
    else:
        outString = val
    return outString

In [ ]:
outdf['in_WaterSourceTypeCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceTypeCV'].unique()

In [ ]:
outdf['in_County'] = outdf.apply(lambda row: ensureEmptyString(row['in_County']), axis=1)
outdf['in_County'].unique()

In [ ]:
outdf['in_SiteName'] = outdf.apply(lambda row: ensureEmptyString(row['in_SiteName']), axis=1)
outdf['in_SiteName'].unique()

In [ ]:
outdf['in_SiteTypeCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_SiteTypeCV']), axis=1)
outdf['in_SiteTypeCV'].unique()

In [ ]:
outdf['in_AllocationLegalStatusCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_AllocationLegalStatusCV']), axis=1)
outdf['in_AllocationLegalStatusCV'].unique()

In [ ]:
outdf['in_AllocationOwner'] = outdf.apply(lambda row: ensureEmptyString(row['in_AllocationOwner']), axis=1)
outdf['in_AllocationOwner'].unique()

In [ ]:
outdf['in_BeneficialUseCategory'] = outdf.apply(lambda row: ensureEmptyString(row['in_BeneficialUseCategory']), axis=1)
outdf['in_BeneficialUseCategory'].unique()

In [ ]:
# in_Latitude
outdf['in_Latitude'] = pd.to_numeric(outdf['in_Latitude'], errors='coerce').fillna("")
outdf['in_Latitude'].unique()

In [ ]:
# in_Longitude
outdf['in_Longitude'] = pd.to_numeric(outdf['in_Longitude'], errors='coerce').fillna("")
outdf['in_Longitude'].unique()

In [ ]:
#Update datatype of Priority Date to fit WaDE 2.0 structure
outdf['in_AllocationPriorityDate'] = pd.to_datetime(outdf['in_AllocationPriorityDate'])
outdf['in_AllocationPriorityDate'] = pd.to_datetime(outdf['in_AllocationPriorityDate'].dt.strftime('%m/%d/%Y'))
outdf['in_AllocationPriorityDate'].unique()

In [ ]:
# Fixing in_AllocationFlow_CFS datatype
outdf['in_AllocationFlow_CFS'] = pd.to_numeric(outdf['in_AllocationFlow_CFS'], errors='coerce').replace(0,"").fillna("")
outdf['in_AllocationFlow_CFS'].unique()

In [ ]:
# Fixing in_AllocationVolume_AF datatype
outdf['in_AllocationVolume_AF'] = pd.to_numeric(outdf['in_AllocationVolume_AF'], errors='coerce').replace(0,"").fillna("")
outdf['in_AllocationVolume_AF'].unique()

In [ ]:
# Creating WaDE Custom water source native ID for easy water source identification
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "wadeID" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceName'] = outdf['in_WaterSourceName']
dfWaterSourceNativeID['in_WaterSourceTypeCV'] = outdf['in_WaterSourceTypeCV']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)
dfWaterSourceNativeID['linkKey'] = dfWaterSourceNativeID['in_WaterSourceName'].astype(str) + dfWaterSourceNativeID['in_WaterSourceTypeCV'].astype(str)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
WaterSourceNativeIDdict = pd.Series(dfWaterSourceNativeID.in_WaterSourceNativeID.values, index=dfWaterSourceNativeID.linkKey.astype(str)).to_dict()
def retrieveWaterSourceNativeID(A, B):
    if (A == '' and B == '') or (pd.isnull(A) and pd.isnull(B)):
        outList = ''
    else:
        colrowValue = str(A).strip() + str(B).strip()
        try:
            outList = WaterSourceNativeIDdict[colrowValue]
        except:
            outList = ''
    return outList

outdf['in_WaterSourceNativeID'] = outdf.apply(lambda row: retrieveWaterSourceNativeID( row['in_WaterSourceName'], row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceNativeID'].unique()

## Shapefile Data
- For attaching gemetry to csv inputs.

In [ ]:
# PoU Shapefile Data
# Shapefile input
dfPoUshapetemp = gpd.read_file('shapefile/BND_SurfaceWaterRights_DNR.zip')
print(len(dfPoUshapetemp))
dfPoUshapetemp.head()

In [ ]:
columnsList = ['in_SiteNativeID', 'geometry']
dfPoUshape = pd.DataFrame(columns=columnsList)
dfPoUshape['in_SiteNativeID'] = "POU" + dfPoUshapetemp['wadeID'].replace("", 0).fillna(0).astype(str)
dfPoUshape['geometry'] = dfPoUshapetemp['geometry']
dfPoUshape = dfPoUshape.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
dfPoUshape.head()

### Exporting to Finished File

In [ ]:
outdf.info()

In [ ]:
outdf

In [ ]:
# Export the output dataframe
outdf.to_csv('Pwr_neMain.zip', compression=dict(method='zip', archive_name='Pwr_neMain.csv'), index=False)  # The output, save as a zip
dfPoUshape.to_csv('P_Geometry.zip', compression=dict(method='zip', archive_name='P_Geometry.csv'), index=False)  # The output geometry.